In [10]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import os

In [18]:
# 将utf-8转换成utf-8-sig
price_data = pd.read_csv('allPrice.csv')
print(price_data.head())

    cityname  zone     name                address  price         lon  \
0  阿坝藏族羌族自治州    茂县     晋茂新园     四川省阿坝藏族羌族自治州茂县晋茂新园   5001  103.840876   
1  阿坝藏族羌族自治州  马尔康市   嘉绒锦绣尚城          四川省德阳市罗江区锦绣尚城   7178  104.505571   
2  阿坝藏族羌族自治州  马尔康市     政法大楼   四川省阿坝藏族羌族自治州马尔康市政法大楼   9008  102.216011   
3  阿坝藏族羌族自治州  马尔康市     绒兴家苑   四川省阿坝藏族羌族自治州马尔康市绒兴家苑   5879  102.220008   
4        巴中市   巴州区  兴合·半山逸城  四川省巴中市巴州区兴合·半山逸城(公交站)   5297  106.749067   

         lat  
0  31.666803  
1  31.306334  
2  31.902818  
3  31.900094  
4  31.869288  


In [20]:
house_data = pd.read_csv('allPrice.csv')
grid = gpd.read_file('grids.geojson')

In [42]:

def cal_max_min_houseprice(grid, data,house_price):
    print("计算price最大最小值")
    print(grid.columns)
    # 为网格数据建立空间索引
    grid = grid.set_geometry('geometry')
    print(grid.columns)
    grid.sindex  # 创建空间索引
    print(grid.sindex)
    # 将数据转换为GeoDataFrame
    data_gdf = gpd.GeoDataFrame(data, geometry=[Point(xy) for xy in zip(data['lon'], data['lat'])])
    data_gdf.crs = grid.crs  # 确保坐标参考系一致

    # 明确指定几何列
    data_gdf = data_gdf.set_geometry('geometry')

    # 遍历网格，使用空间索引进行筛选
    for i, row in grid.iterrows():
        # 获取当前网格的几何边界
        grid_polygon = row['geometry']

        # # 打印网格的边界信息，调试用
        # print(f"处理第 {i} 个网格，边界：", grid_polygon.bounds)

        # 使用空间索引查找可能在当前网格内的点
        possible_matches_index = list(data_gdf.sindex.intersection(grid_polygon.bounds))
        # print(f"可能的匹配项索引：{possible_matches_index}")
        
        # 提取这些候选数据
        possible_matches = data_gdf.iloc[possible_matches_index]
        # print("possible matches:",possible_matches)

        # 筛选在当前网格内的点
        filtered_data = possible_matches[possible_matches.geometry.within(grid_polygon)]

        if not filtered_data.empty:
            # 计算人口密度的平均值
            mean_value = np.mean(filtered_data['price'])
            house_price.append(mean_value)
        else:
            house_price.append(0)
        
        if i % 100000 == 0:
            print(f"已完成 {i} 个网格")
    
    return house_price

In [47]:
house_price = []

house_price = cal_max_min_houseprice(grid, house_data, house_price)
# 去除NaN值后计算最大最小值
# house_price = [x for x in house_price if not np.isnan(x)]
# print(population_density)
print("房价最大值：",max(house_price))#55852.333333333336
print("房价最小值：",min(house_price))#0

计算price最大最小值
Index(['geometry'], dtype='object')
Index(['geometry'], dtype='object')
已完成 0 个网格
已完成 100000 个网格
已完成 200000 个网格
已完成 300000 个网格
已完成 400000 个网格
已完成 500000 个网格
已完成 600000 个网格
已完成 700000 个网格
已完成 800000 个网格
已完成 900000 个网格
房价最大值： 55852.333333333336
房价最小值： 0


In [48]:
print(len(house_price))

989000


In [49]:
np.save('house_price.npy', house_price)

In [50]:
# min-max标准化
def min_max_scaler(data):
    data = (data - np.min(data)) / (np.max(data) - np.min(data))
    return data

In [51]:
scale_house_price = min_max_scaler(house_price)
# 保存到文件
np.save('scale_house_price.npy',scale_house_price)

In [30]:
loaded_scale_house_price = np.load('scale_house_price.npy')
print(loaded_scale_house_price[0:10])

[0.02407885 0.0299892  0.05226548 0.04780402 0.04329912 0.01357053
 0.02278509 0.02757853 0.09408158 0.04624654]


In [40]:
print(len(scale_house_price))

3120


In [53]:
house_price_prob = []
sum_house_price = np.sum(scale_house_price)
# 计算概率矩阵
for i in range(len(scale_house_price)):
    house_price_prob.append(scale_house_price[i] / sum_house_price)



In [62]:
print(len(house_price_prob))

989000


In [56]:
np.save('prob_house_price.npy',house_price_prob)

In [17]:


# 定义文件路径
path = "D:\\系统默认\\桌面\\邮政\\数据调研\\newPrice"

# 初始化一个空的 DataFrame
combined_df = pd.DataFrame()

# 遍历所有文件
for file in os.listdir(path):
    if file.endswith('.csv'):
        file_path = os.path.join(path, file)
        try:
            # 尝试使用 utf-8 编码读取 CSV 文件
            df = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            # 如果失败，尝试使用 latin1 编码读取 CSV 文件
            df = pd.read_csv(file_path, encoding='latin1')
        # 将数据追加到 combined_df
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# 保存合并后的数据到新的 CSV 文件，使用带有 BOM 的 UTF-8 编码
combined_df.to_csv('combined_data.csv', encoding='utf-8-sig', index=False)


In [ ]:
import os
import pandas as pd

# 定义文件路径
path = "D:\\系统默认\\桌面\\邮政\\数据调研\\Price"

# 初始化一个空的 DataFrame
combined_df = pd.DataFrame()

# 遍历所有文件
for file in os.listdir(path):
    if file.endswith('.csv'):
        file_path = os.path.join(path, file)
        # 读取 CSV 文件
        df = pd.read_csv(file_path)
        # 将数据追加到 combined_df
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# 保存合并后的数据到新的 CSV 文件，使用带有 BOM 的 UTF-8 编码
combined_df.to_csv('combined_data.csv', encoding='utf-8-sig', index=False)

In [ ]:
import os
import pandas as pd

# 定义文件路径
path = "D:\\系统默认\\桌面\\邮政\\数据调研\\Price"

# 初始化一个空的 DataFrame
combined_df = pd.DataFrame()

# 遍历所有文件
for file in os.listdir(path):
    if file.endswith('.csv'):
        file_path = os.path.join(path, file)
        # 读取 CSV 文件
        df = pd.read_csv(file_path)
        # 将数据追加到 combined_df
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# 保存合并后的数据到新的 CSV 文件，使用带有 BOM 的 UTF-8 编码
combined_df.to_csv('combined_data.csv', encoding='utf-8-sig', index=False)

In [ ]:
import os
import pandas as pd

# 定义文件路径
path = "D:\\系统默认\\桌面\\邮政\\数据调研\\Price"

# 初始化一个空的 DataFrame
combined_df = pd.DataFrame()

# 遍历所有文件
for file in os.listdir(path):
    if file.endswith('.csv'):
        file_path = os.path.join(path, file)
        # 读取 CSV 文件
        df = pd.read_csv(file_path)
        # 将数据追加到 combined_df
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# 保存合并后的数据到新的 CSV 文件，使用带有 BOM 的 UTF-8 编码
combined_df.to_csv('combined_data.csv', encoding='utf-8-sig', index=False)

In [57]:
scale_poi_density = np.load('scale_poi_density.npy')
scale_poi_diversity = np.load('scale_poi_diversity.npy')

In [58]:
poi_density_prob = []
poi_diversity_prob = []
sum_poi_density = np.sum(scale_poi_density)
sum_poi_diversity = np.sum(scale_poi_diversity)
# 计算概率矩阵
for i in range(len(scale_poi_density)):
    poi_density_prob.append(scale_poi_density[i] / sum_poi_density)

for i in range(len(scale_poi_diversity)):
    poi_diversity_prob.append(scale_poi_diversity[i] / sum_poi_diversity)

np.save('prob_poi_density.npy', poi_density_prob)
np.save('prob_poi_diversity.npy', poi_diversity_prob)

In [61]:
# 有多少0
print(poi_density_prob.count(0))

947961


In [68]:
# 计算信息熵
def cal_entropy(prob):
    entropy = 0
    for i in range(len(prob)):
        if prob[i] != 0:
            entropy -= prob[i] * np.log(prob[i])
    # 变为0-1之间
    entropy /= np.log(len(prob))
    return entropy

In [69]:
hosue_entropy = cal_entropy(house_price_prob)
print(hosue_entropy) #0.5709951600935983

0.5709951600935983


In [70]:
poi_density_entropy = cal_entropy(poi_density_prob)
print(poi_density_entropy) #0.6428892367629705
poi_diversity_entropy = cal_entropy(poi_diversity_prob)
print(poi_diversity_entropy) #0.6915434317121325

0.6428892367629705
0.6915434317121325
